In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="bcLcEpxMIQLbehAgWUQa")
project = rf.workspace("roboflow-100").project("x-ray-rheumatology")
version = project.version(2)
dataset = version.download("yolov5")


In [2]:
!pip install -q roboflow ultralytics
!pip install -q scikit-learn matplotlib seaborn pandas opencv-python-headless


In [ ]:
# Import các thư viện
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import torch
import cv2
import pandas as pd
from roboflow import Roboflow
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix

# Thiết lập random seed để tái tạo lại kết quả
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Tải dữ liệu từ Roboflow
rf = Roboflow(api_key="bcLcEpxMIQLbehAgWUQa")
project = rf.workspace("roboflow-100").project("x-ray-rheumatology")
version = project.version(2)
dataset = version.download("yolov5")

# Lấy đường dẫn hiện tại
# DATA_DIR = Path(os.getcwd()) # This line was causing the error
#The dataset is downloaded into a folder with the same name as the project and version
# Instead of version.version_number, use version.id to get the version number
DATA_DIR = "/content/x-ray-rheumatology-2"
print(f"Dữ liệu được tải về thư mục: {DATA_DIR}")

# Hiển thị thông tin về dữ liệu
yaml_file = "/content/x-ray-rheumatology-2/data.yaml"
print(f"Nội dung file data.yaml:\n{open(yaml_file).read()}")

# Kiểm tra số lượng ảnh trong mỗi tập
# Convert DATA_DIR to a Path object
DATA_DIR = Path(DATA_DIR)
train_dir = DATA_DIR / 'train' / 'images' # Use / for joining paths with Path objects
val_dir = DATA_DIR / 'valid' / 'images'
test_dir = DATA_DIR / 'test' / 'images'

print(f"Số lượng ảnh trong tập train: {len(list(train_dir.glob('*.jpg')))}")
print(f"Số lượng ảnh trong tập validation: {len(list(val_dir.glob('*.jpg')))}")
print(f"Số lượng ảnh trong tập test: {len(list(test_dir.glob('*.jpg')))}")

# Hiển thị một số ảnh mẫu từ tập huấn luyện
def plot_sample_images(image_dir, num_samples=3):
    plt.figure(figsize=(15, 5))
    sample_images = list(image_dir.glob('*.jpg'))[:num_samples]

    for i, img_path in enumerate(sample_images):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Đọc nhãn tương ứng
        label_path = str(img_path).replace('images', 'labels').replace('.jpg', '.txt')

        plt.subplot(1, num_samples, i+1)
        plt.imshow(img)
        plt.title(f"Sample {i+1}")
        plt.axis('off')

    plt.tight_layout()
    plt.savefig('sample_images.png')
    plt.show()

plot_sample_images(train_dir)

# Phân tích phân bố của các lớp
def analyze_class_distribution(label_dir, class_names):
    class_counts = {name: 0 for name in class_names}

    # Đếm số lượng object cho mỗi lớp
    for label_file in label_dir.glob('*.txt'):
        with open(label_file, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])
                class_counts[class_names[class_id]] += 1

    # Vẽ biểu đồ phân bố
    plt.figure(figsize=(12, 6))
    sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
    plt.xticks(rotation=45, ha='right')
    plt.title('Phân bố các lớp trong tập dữ liệu')
    plt.tight_layout()
    plt.savefig('class_distribution.png')
    plt.show()

    return class_counts

In [ ]:
import yaml
with open(yaml_file, 'r') as f:
    data_yaml = yaml.safe_load(f)
    class_names = data_yaml['names']

train_labels_dir = DATA_DIR / 'train' / 'labels'
class_counts = analyze_class_distribution(train_labels_dir, class_names)
print("Số lượng đối tượng cho mỗi lớp:", class_counts)


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


In [ ]:
%cd yolov5


In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data {DATA_DIR}/data.yaml --weights yolov5s.pt --name x-ray-rheumatology


In [5]:
weights_path = Path('/content/yolov5/runs/train/x-ray-rheumatology-continued/weights/best.pt')


In [ ]:
!python train.py --img 640 --batch 16 --epochs 200 --data {DATA_DIR}/data.yaml --weights runs/train/x-ray-rheumatology3/weights/best.pt --name x-ray-rheumatology-continued

In [ ]:
!python val.py --weights {weights_path} --data {DATA_DIR}/data.yaml --img 640 --task test


In [ ]:
!python detect.py --weights {weights_path} --source {test_dir} --conf 0.25 --save-txt --save-conf


In [ ]:
import os
import random
import numpy as np
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
import shutil

def load_yolo_annotations(annotation_path):
    """Đọc annotation định dạng YOLO từ file txt"""
    boxes = []
    if os.path.exists(annotation_path):
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = int(parts[0])
                    x_center, y_center = float(parts[1]), float(parts[2])
                    width, height = float(parts[3]), float(parts[4])
                    boxes.append([class_id, x_center, y_center, width, height])
    return np.array(boxes)

def yolo_to_pascal_voc(yolo_boxes, img_width, img_height):
    """Chuyển đổi từ định dạng YOLO sang Pascal VOC (x_min, y_min, x_max, y_max)"""
    pascal_boxes = []
    for box in yolo_boxes:
        class_id = box[0]
        x_center, y_center = box[1], box[2]
        width, height = box[3], box[4]

        x_min = int((x_center - width/2) * img_width)
        y_min = int((y_center - height/2) * img_height)
        x_max = int((x_center + width/2) * img_width)
        y_max = int((y_center + height/2) * img_height)

        pascal_boxes.append([x_min, y_min, x_max, y_max, class_id])

    return np.array(pascal_boxes)

def pascal_voc_to_yolo(pascal_boxes, img_width, img_height):
    """Chuyển đổi từ định dạng Pascal VOC sang YOLO"""
    yolo_boxes = []
    for box in pascal_boxes:
        x_min, y_min, x_max, y_max = box[0], box[1], box[2], box[3]
        class_id = box[4]

        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        yolo_boxes.append([class_id, x_center, y_center, width, height])

    return np.array(yolo_boxes)

def save_yolo_annotations(boxes, output_path):
    """Lưu annotation định dạng YOLO vào file txt"""
    with open(output_path, 'w') as f:
        for box in boxes:
            box = [str(round(float(x), 6)) for x in box]
            f.write(' '.join(box) + '\n')

def apply_augmentation(image, bboxes, augmentation):
    """Áp dụng tăng cường dữ liệu cho ảnh và bounding boxes"""
    # Chuyển đổi từ YOLO format sang Pascal VOC format
    height, width = image.shape[:2]
    pascal_boxes = []
    class_labels = []

    if len(bboxes) > 0:
        for box in bboxes:
            class_id = int(box[0])
            x_center, y_center = box[1], box[2]
            w, h = box[3], box[4]

            x_min = int((x_center - w/2) * width)
            y_min = int((y_center - h/2) * height)
            x_max = int((x_center + w/2) * width)
            y_max = int((y_center + h/2) * height)

            pascal_boxes.append([x_min, y_min, x_max, y_max])
            class_labels.append(class_id)

        # Áp dụng augmentation
        transformed = augmentation(
            image=image,
            bboxes=pascal_boxes,
            class_labels=class_labels
        )

        # Lấy kết quả
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_labels = transformed['class_labels']

        # Chuyển đổi bboxes trở lại định dạng YOLO
        transformed_yolo = []
        h, w = transformed_image.shape[:2]

        for i, (x_min, y_min, x_max, y_max) in enumerate(transformed_bboxes):
            x_center = ((x_min + x_max) / 2) / w
            y_center = ((y_min + y_max) / 2) / h
            width_norm = (x_max - x_min) / w
            height_norm = (y_max - y_min) / h

            transformed_yolo.append([transformed_labels[i], x_center, y_center, width_norm, height_norm])

        return transformed_image, np.array(transformed_yolo)

    return image, np.array(bboxes)

def create_augmentation_pipeline(aug_type='default'):
    """Tạo pipeline augmentation cho ảnh X-quang"""
    if aug_type == 'default':
        return A.Compose([
            A.RandomRotate90(p=0.2),
            A.HorizontalFlip(p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
            A.GaussNoise(var=15.0, p=0.3),  # Sửa var_limit thành var
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    elif aug_type == 'medical':
        # Augmentation đặc biệt cho ảnh y tế
        return A.Compose([
            A.Affine(scale=(0.95, 1.05), translate_percent=(0.02, 0.02), rotate=(-5, 5), p=0.5),  # Thay ShiftScaleRotate
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.2, p=0.5),
            A.GaussianBlur(blur_limit=(1, 3), p=0.2),
            A.CLAHE(clip_limit=1.5, p=0.3),
            A.Equalize(p=0.2),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    elif aug_type == 'strong':
        # Augmentation mạnh khi dataset nhỏ
        return A.Compose([
            A.OneOf([
                A.RandomRotate90(p=1.0),
                A.Rotate(limit=10, p=1.0),
                A.Affine(scale=(0.9, 1.1), translate_percent=(0.05, 0.05), rotate=(-15, 15), p=1.0),  # Thay ShiftScaleRotate
            ], p=0.5),
            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
                A.CLAHE(clip_limit=2.0, p=1.0),
                A.Equalize(p=1.0),
            ], p=0.5),
            A.OneOf([
                A.GaussNoise(var=30.0, p=1.0),  # Sửa var_limit thành var
                A.GaussianBlur(blur_limit=(1, 3), p=1.0),
                A.MotionBlur(blur_limit=3, p=1.0),
            ], p=0.3),
            A.OneOf([
                A.ElasticTransform(alpha=1, sigma=50, p=1.0),  # Bỏ alpha_affine
                A.GridDistortion(num_steps=5, distort_limit=0.3, p=1.0),
                A.OpticalDistortion(distort_limit=0.3, shift_limit=0.5, p=1.0),
            ], p=0.2),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    elif aug_type == 'ra_specific':
        # Tăng cường đặc biệt cho viêm khớp dạng thấp
        return A.Compose([
            # Biến đổi nhẹ về vị trí và góc quay (do ảnh X-quang thường được chụp ở các góc tiêu chuẩn)
            A.Affine(scale=(0.95, 1.05), translate_percent=(0.03, 0.03), rotate=(-7, 7), p=0.5),  # Thay ShiftScaleRotate

            # Điều chỉnh tương phản và độ sáng để mô phỏng các điều kiện chụp khác nhau
            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.2, p=1.0),
                A.CLAHE(clip_limit=2.0, tile_grid_size=(4, 4), p=1.0),
                A.Equalize(p=1.0),
            ], p=0.7),

            # Mô phỏng điều kiện chất lượng ảnh khác nhau
            A.OneOf([
                A.GaussNoise(var=20.0, p=1.0),  # Sửa var_limit thành var
                A.GaussianBlur(blur_limit=(1, 2), p=1.0),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0),
            ], p=0.3),

            # Thêm một số biến đổi rất nhẹ về hình thái để mô phỏng góc chụp khác nhau
            A.OneOf([
                A.ElasticTransform(alpha=0.5, sigma=25, p=1.0),  # Bỏ alpha_affine
                A.GridDistortion(num_steps=3, distort_limit=0.1, p=1.0),
            ], p=0.1),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

    else:
        # Default augmentation
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

def augment_dataset(input_dir, output_dir, class_names, aug_types=['default'],
                   num_augmentations=2, visualize=True):
    """Tạo bộ dữ liệu tăng cường từ bộ gốc"""
    # Tạo thư mục đầu ra
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)

    # Tìm tất cả các file ảnh
    image_paths = []
    for ext in ['.jpg', '.jpeg', '.png']:
        image_paths.extend(list(Path(os.path.join(input_dir, 'images')).glob(f'*{ext}')))

    # Tạo pipeline tăng cường
    augmentations = {}
    for aug_type in aug_types:
        augmentations[aug_type] = create_augmentation_pipeline(aug_type)

    # Sao chép dữ liệu gốc sang thư mục đầu ra
    for img_path in image_paths:
        img_name = img_path.name
        label_path = os.path.join(input_dir, 'labels', img_name.rsplit('.', 1)[0] + '.txt')

        # Sao chép ảnh gốc
        shutil.copy(str(img_path), os.path.join(output_dir, 'images', img_name))

        # Sao chép nhãn gốc nếu tồn tại
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(output_dir, 'labels', img_name.rsplit('.', 1)[0] + '.txt'))

    # Tạo dữ liệu tăng cường
    for img_path in image_paths:
        img_name = img_path.name
        base_name = img_name.rsplit('.', 1)[0]
        img_ext = img_name.rsplit('.', 1)[1]
        label_path = os.path.join(input_dir, 'labels', base_name + '.txt')

        # Đọc ảnh
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Đọc annotation
        bboxes = load_yolo_annotations(label_path)

        # Áp dụng tăng cường
        for aug_idx in range(num_augmentations):
            for aug_type, aug_pipeline in augmentations.items():
                try:
                    # Áp dụng pipeline tương ứng
                    aug_image, aug_bboxes = apply_augmentation(image.copy(), bboxes.copy(), aug_pipeline)

                    # Tạo tên file mới
                    new_img_name = f"{base_name}_{aug_type}_{aug_idx}.{img_ext}"
                    new_label_name = f"{base_name}_{aug_type}_{aug_idx}.txt"

                    # Lưu ảnh mới
                    aug_image_rgb = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(os.path.join(output_dir, 'images', new_img_name), aug_image_rgb)

                    # Lưu annotation mới
                    if len(aug_bboxes) > 0:
                        save_yolo_annotations(aug_bboxes, os.path.join(output_dir, 'labels', new_label_name))

                    # Hiển thị kết quả tăng cường nếu cần
                    if visualize and aug_idx == 0:
                        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

                        # Hiển thị ảnh gốc với bboxes
                        ax[0].imshow(image)
                        ax[0].set_title(f"Ảnh gốc")
                        height, width = image.shape[:2]
                        for box in bboxes:
                            class_id, x_center, y_center, w, h = box
                            x1 = int((x_center - w/2) * width)
                            y1 = int((y_center - h/2) * height)
                            x2 = int((x_center + w/2) * width)
                            y2 = int((y_center + h/2) * height)

                            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2)
                            ax[0].add_patch(rect)
                            if class_names and int(class_id) < len(class_names):
                                ax[0].text(x1, y1, class_names[int(class_id)], color='white',
                                         bbox=dict(facecolor='red', alpha=0.5))

                        # Hiển thị ảnh tăng cường với bboxes
                        ax[1].imshow(aug_image)
                        ax[1].set_title(f"Tăng cường: {aug_type}")
                        height, width = aug_image.shape[:2]
                        for box in aug_bboxes:
                            class_id, x_center, y_center, w, h = box
                            x1 = int((x_center - w/2) * width)
                            y1 = int((y_center - h/2) * height)
                            x2 = int((x_center + w/2) * width)
                            y2 = int((y_center + h/2) * height)

                            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='blue', linewidth=2)
                            ax[1].add_patch(rect)
                            if class_names and int(class_id) < len(class_names):
                                ax[1].text(x1, y1, class_names[int(class_id)], color='white',
                                         bbox=dict(facecolor='blue', alpha=0.5))
                except Exception as e:
                    print(f"Lỗi khi xử lý ảnh {img_name} với augmentation {aug_type}: {str(e)}")
                    continue

    # Tạo file data.yaml cho bộ dữ liệu mới
    yaml_content = {
        'train': './images',
        'val': './images',  # Thường nên tách riêng tập validation
        'test': './images',  # Thường nên tách riêng tập test
        'nc': len(class_names),
        'names': class_names
    }

    with open(os.path.join(output_dir, 'data.yaml'), 'w') as f:
        yaml.dump(yaml_content, f)

    # Báo cáo kết quả
    original_count = len(image_paths)
    augmented_count = original_count * (1 + len(aug_types) * num_augmentations)

    print(f"Đã tạo bộ dữ liệu tăng cường:")
    print(f"- Số lượng ảnh gốc: {original_count}")
    print(f"- Số lượng ảnh sau tăng cường: {augmented_count}")
    print(f"- Tỉ lệ tăng cường: {augmented_count / original_count:.2f}x")
    print(f"- Các phương pháp tăng cường đã sử dụng: {', '.join(aug_types)}")
    print(f"- Bộ dữ liệu mới được lưu tại: {output_dir}")

    return output_dir

class_names = ['artefact', 'distal phalanges', 'fifth metacarpal bone', 'first metacarpal bone',
              'fourth metacarpal bone', 'intermediate phalanges', 'proximal phalanges', 'radius',
              'second metacarpal bone', 'soft tissue calcination', 'third metacarpal bone', 'ulna']

# Tạo bộ dữ liệu tăng cường với nhiều loại kỹ thuật
augment_dataset(
    input_dir='/content/x-ray-rheumatology-2/train',
    output_dir='/content/augmented_dataset',
    class_names=class_names,
    aug_types=['default', 'medical', 'ra_specific'],  # Sử dụng nhiều loại augmentation
    num_augmentations=2,    # Tăng số lượng ảnh cho mỗi loại
    visualize=True
)

In [ ]:
import os
import shutil
from pathlib import Path

# Thư mục nguồn
aug_dataset_dir = '/content/augmented_dataset'
images_dir = os.path.join(aug_dataset_dir, 'images')
labels_dir = os.path.join(aug_dataset_dir, 'labels')

# Tạo các thư mục cần thiết
train_img_dir = os.path.join(aug_dataset_dir, 'train', 'images')
train_label_dir = os.path.join(aug_dataset_dir, 'train', 'labels')
val_img_dir = os.path.join(aug_dataset_dir, 'valid', 'images')
val_label_dir = os.path.join(aug_dataset_dir, 'valid', 'labels')
test_img_dir = os.path.join(aug_dataset_dir, 'test', 'images')
test_label_dir = os.path.join(aug_dataset_dir, 'test', 'labels')

# Tạo thư mục nếu chưa tồn tại
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(test_img_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)

# Lấy danh sách tất cả các ảnh
all_images = list(Path(images_dir).glob('*.jpg'))
import random
random.shuffle(all_images)

# Chia tập dữ liệu (80% train, 10% validation, 10% test)
total = len(all_images)
train_count = int(total * 0.8)
val_count = int(total * 0.1)

train_images = all_images[:train_count]
val_images = all_images[train_count:train_count+val_count]
test_images = all_images[train_count+val_count:]

# Di chuyển các file vào thư mục tương ứng
for img_path in train_images:
    img_name = img_path.name
    label_name = img_name.rsplit('.', 1)[0] + '.txt'
    label_path = os.path.join(labels_dir, label_name)

    shutil.copy(str(img_path), os.path.join(train_img_dir, img_name))
    if os.path.exists(label_path):
        shutil.copy(label_path, os.path.join(train_label_dir, label_name))

for img_path in val_images:
    img_name = img_path.name
    label_name = img_name.rsplit('.', 1)[0] + '.txt'
    label_path = os.path.join(labels_dir, label_name)

    shutil.copy(str(img_path), os.path.join(val_img_dir, img_name))
    if os.path.exists(label_path):
        shutil.copy(label_path, os.path.join(val_label_dir, label_name))

for img_path in test_images:
    img_name = img_path.name
    label_name = img_name.rsplit('.', 1)[0] + '.txt'
    label_path = os.path.join(labels_dir, label_name)

    shutil.copy(str(img_path), os.path.join(test_img_dir, img_name))
    if os.path.exists(label_path):
        shutil.copy(label_path, os.path.join(test_label_dir, label_name))

print(f"Đã tổ chức lại bộ dữ liệu:")
print(f"- Train: {len(train_images)} ảnh")
print(f"- Validation: {len(val_images)} ảnh")
print(f"- Test: {len(test_images)} ảnh")

# Cập nhật file data.yaml với đường dẫn tuyệt đối
yaml_path = os.path.join(aug_dataset_dir, 'data.yaml')
with open(yaml_path, 'r') as f:
    yaml_content = yaml.safe_load(f)

yaml_content['train'] = '/content/augmented_dataset/train/images'
yaml_content['val'] = '/content/augmented_dataset/valid/images'
yaml_content['test'] = '/content/augmented_dataset/test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)

print("Đã cập nhật file data.yaml với đường dẫn tuyệt đối")

In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data /content/augmented_dataset/data.yaml --weights yolov5s.pt --name ra_augmented